In [4]:
# Import Packages
import streamlit as st
import pyperclip
import heapq
import nltk
import re

from newspaper import Article

In [3]:
import pandas as pd
df=pd.DataFrame(['Text to copy'])
df.to_clipboard(index=False,header=False)

In [ ]:
# Pull Article Information

url = pyperclip.paste()

In [ ]:
article = Article(url)
article.download()
article.parse()
article.nlp()

In [ ]:
article_title = article.title
print(article_title)

In [ ]:
source_url = article.url
print(source_url)

In [ ]:
article_summary = article.summary
print(article_summary)

In [ ]:
# Removing Square Brackets and Extra Spaces
article_text = re.sub(r'\[[0-9]*\]', ' ', article_summary)
print(article_text)

In [ ]:
article_text = re.sub(r'\s+', ' ', article_summary)
print(article_text)

In [ ]:
# Removing special characters and digits
#formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text)
#print(formatted_article_text)

In [ ]:
#formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
#print(formatted_article_text)

In [ ]:
sentence_list = nltk.sent_tokenize(article_text)
print(sentence_list)

In [ ]:
len(sentence_list)

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

In [ ]:
word_frequencies = {}
for word in nltk.word_tokenize(article_text):
    if len(word) >= 7:
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
                
print(word_frequencies)

In [ ]:
sorted_values = sorted(word_frequencies.values(), reverse=True)  # Sort the values

print(sorted_values)

In [ ]:
sorted_dict = {}

for i in sorted_values:
    for k in word_frequencies.keys():
        if word_frequencies[k] == i:
            sorted_dict[k] = word_frequencies[k]
            break
            
print(sorted_dict)

In [ ]:
sorted_list = list(sorted_dict.keys())
print(sorted_list)

In [ ]:
maximum_freq = max(word_frequencies.values())
print(maximum_freq)

In [ ]:
for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word] / maximum_freq)
    
print(word_frequencies)

In [ ]:
sentence_scores = {}

for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
                    
print(sentence_scores)

In [ ]:
summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

print(sentence_scores)

In [ ]:
summary = ' '.join(summary_sentences)

print(summary)

In [ ]:
if 'BREAKING' in article_title:
    default_text = '''
🚨 {}

🔑 SUMMARY: {}

🔗 {}
            '''.format(article_title, summary, source_url)
else:
    default_text = '''
📰 {}

🔑 SUMMARY: {}

🔗 {}
            '''.format(article_title, summary, source_url)
    
print(default_text)

<hr>
<hr>
<hr>
<hr>
<hr>
<hr>

In [5]:
%cd '/Users/josephrosas/PycharmProjects/media-numbers'

! python newsscraper.py NewsPapers.json

/Users/josephrosas/PycharmProjects/media-numbers
1 articles downloaded from cnn, url: http://rss.cnn.com/~r/rss/cnn_topstories/~3/YTkKaI0H22o/index.html
2 articles downloaded from cnn, url: http://rss.cnn.com/~r/rss/cnn_topstories/~3/7TNeNF_jVkc/far-right-extremist-groups-shift-vaccine-misinformation-burghart-intv-nr-vpx.cnn
3 articles downloaded from cnn, url: http://rss.cnn.com/~r/rss/cnn_topstories/~3/_FqBKjclf0U/index.html
4 articles downloaded from cnn, url: http://rss.cnn.com/~r/rss/cnn_topstories/~3/Zmq85Qb5dsQ/index.html
5 articles downloaded from cnn, url: http://rss.cnn.com/~r/rss/cnn_topstories/~3/oWUgrBlMvdM/pfizer-vaccine-protection-length-watt-dnt-tsr-vpx.cnn
1 articles downloaded from nyt, url: https://www.nytimes.com/2021/04/05/us/politics/gayle-smith-covid-vaccine.html
2 articles downloaded from nyt, url: https://www.nytimes.com/2021/04/05/us/politics/supreme-court-trump-twitter.html
3 articles downloaded from nyt, url: https://www.nytimes.com/2021/04/05/us/politics/he

1 articles downloaded from fte, url: https://fivethirtyeight.com/features/why-being-anti-media-is-now-part-of-the-gop-identity/
2 articles downloaded from fte, url: https://fivethirtyeight.com/features/americans-oppose-many-voting-restrictions-but-not-voter-id-laws/
3 articles downloaded from fte, url: https://fivethirtyeight.com/videos/the-2021-elections-that-will-show-us-where-the-parties-are-headed/
4 articles downloaded from fte, url: https://fivethirtyeight.com/features/politics-podcast-elections-to-watch-in-2021/
5 articles downloaded from fte, url: https://fivethirtyeight.com/features/why-the-recent-violence-against-asian-americans-may-solidify-their-support-of-democrats/
1 articles downloaded from wayne, url: https://www.waynedupree.com/2021/04/family-mask-spirit-airlines/
2 articles downloaded from wayne, url: https://www.waynedupree.com/2021/04/tilsi-fox-news-republican/
3 articles downloaded from wayne, url: https://www.waynedupree.com/2021/04/scotus-save-nominee-process/
4 

In [6]:
import json
import jmespath
import pandas as pd
import pyperclip
import heapq
import nltk
import re

from glom import glom
from ast import literal_eval
from pandas import json_normalize
from newspaper import Article


def f2(json):
    df = pd.DataFrame(json)
    df = df.iloc[:, 2:].copy()
    df = pd.concat([pd.DataFrame(json_normalize(x)) for x in df['articles']],
                   ignore_index=True)
    return df

# Load json format data
with open('scraped_articles.json', 'r') as f:
    data = json.loads(f.read())

recs = data['newspapers']

In [7]:
cnn_df = f2(recs['cnn']) # CNN
fox_df = f2(recs['fox']) # Fox News
trs_df = f2(recs['trs']) # The Right Scoop
epoch_df = f2(recs['epoch']) # The Epoch Times
blaze_df = f2(recs['blaze']) # BlazeTV
tgp_df = f2(recs['tgp']) # The Gateway Pundit
trend_df = f2(recs['trend']) # Trending Politics
wayne_df = f2(recs['wayne']) # Wayne Dupree
tnr_df = f2(recs['tnr']) # The National Review
fte_df = f2(recs['fte']) # FiveThirtyEight
hotair_df = f2(recs['hotair']) # Hot Air
rds_df = f2(recs['rds']) # Red State
nyt_df = f2(recs['nyt']) # Red State
politicus_usa_df = f2(recs['politicus_usa']) # Red State

#cdm_json = recs['cdm']
#cdm_df = f2(cdm_json)

In [8]:
dataframe = tgp_df.copy()

url = dataframe['link'][0]

sentence_length = 1

In [10]:
# Get Summary 

article = Article(url)
article.download()
article.parse()
article.nlp()

article_title = article.title
source_url = article.url
article_text = article.text
article_text_formatted = re.sub(r'\s+', ' ', article_text)
sentence_list = nltk.sent_tokenize(article_text_formatted)
stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}
for word in nltk.word_tokenize(article_text_formatted):
    if len(word) >= 5:
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

sorted_values = sorted(word_frequencies.values(),
                       reverse=True)  # Sort the values
sorted_dict = {}

for i in sorted_values:
    for k in word_frequencies.keys():
        if word_frequencies[k] == i:
            sorted_dict[k] = word_frequencies[k]
            break

sorted_list = list(sorted_dict.keys())

maximum_freq = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word] / maximum_freq)

sentence_scores = {}

for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

summary_sentences = heapq.nlargest(sentence_length, sentence_scores, key=sentence_scores.get)
summary = ' '.join(summary_sentences)

tag_list = []
for w in sorted_list:
    w = w.capitalize()
    tag = ' #' + w
    tag_list.append('{}'.format(tag))
tags = ''.join(tag_list)


if 'BREAKING' in article_title:
    default_text = '''
🚨 {}

🔑 SUMMARY: {}

{} {} {} {}

🔗 {}
            '''.format(article_title, summary, source_url)
else:
    default_text = '''
📰 {}

🔑 SUMMARY: {}

{}

🔗 {}
            '''.format(article_title, summary, tags, source_url)

#pyperclip.copy(default_text)
import pandas as pd
df=pd.DataFrame([default_text])
df.to_clipboard(index=False,header=False)

print(default_text)


📰 Justice Thomas Drops The Hammer on Big Tech In Epic Defense For Conservatives in America

🔑 SUMMARY: “Even if digital platforms are not close enough to common carriers, legislatures might still be able to treat digital platforms like places of public accommodation.

 #Platforms #Public #Thomas #Accommodation #Common #Twitter #First #Places #Supreme

🔗 https://trendingpolitics.com/justice-thomas-drops-the-hammer-on-big-tech-in-epic-defense-for-conservatives-in-america/?utm_source=rss&utm_medium=rss&utm_campaign=justice-thomas-drops-the-hammer-on-big-tech-in-epic-defense-for-conservatives-in-america
            


In [ ]:

📰 Justice Thomas Drops The Hammer on Big Tech In Epic Defense For Conservatives in America

🔑 SUMMARY: “Even if digital platforms are not close enough to common carriers, legislatures might still be able to treat digital platforms like places of public accommodation.

 #Platforms #Public #Thomas #Accommodation #Common #Twitter #First #Places #Supreme

🔗 https://trendingpolitics.com/justice-thomas-drops-the-hammer-on-big-tech-in-epic-defense-for-conservatives-in-america/?utm_source=rss&utm_medium=rss&utm_campaign=justice-thomas-drops-the-hammer-on-big-tech-in-epic-defense-for-conservatives-in-america

In [11]:
import pyp3rclip

In [12]:
pyp3rclip.copy('The text to be copied to the clipboard.')

'https://www.theepochtimes.com/supreme-court-throws-out-lawsuit-over-trump-blocking-critics-on-twitter_3762613.html'